# 04 — Comparativa de Modelos

**Proyecto:** MLY0100 — Pipeline Diabetes con Kedro  
**Autor:** Antonio Sepúlveda  
**Fecha:** 2025

---
Este notebook evalúa **múltiples modelos de clasificación** para determinar cuál ofrece el mejor desempeño en la predicción de **diabetes**.

Se realiza la comparación usando:
- RandomForest (modelo actual del pipeline)
- Logistic Regression
- Support Vector Machine (SVM)
- KNN
- Gradient Boosting

Finalmente, se elige el **mejor modelo**, validando métricas y visualizaciones adicionales.

# 📚 1. Importación de Librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# 📁 2. Carga del Dataset Limpio

Se cargan los datos ya limpios desde el pipeline Kedro:

In [ ]:
df = pd.read_csv("../data/02_intermediate/diabetes_cleaned.csv")
df.head()

# 🎯 3. Preparación de Variables

Separación entre variables predictoras **X** y variable objetivo **y (Outcome)**.

In [ ]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

# 🤖 4. Definición de Modelos a Comparar

In [ ]:
modelos = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=500),
    "SVM": SVC(kernel="rbf", probability=True),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "GradientBoosting": GradientBoostingClassifier(random_state=42)
}

# 📊 5. Entrenamiento y Evaluación

In [ ]:
resultados = []

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    pred = modelo.predict(X_test)

    resultados.append({
        "Modelo": nombre,
        "Accuracy": accuracy_score(y_test, pred),
        "Precision": precision_score(y_test, pred),
        "Recall": recall_score(y_test, pred),
        "F1-score": f1_score(y_test, pred)
    })

df_resultados = pd.DataFrame(resultados)
df_resultados

# 📊 6. Comparación Gráfica de Métricas

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=df_resultados.melt(id_vars="Modelo"), x="variable", y="value", hue="Modelo")
plt.title("Comparación de Métricas entre Modelos", fontsize=14)
plt.xlabel("Métrica")
plt.ylabel("Valor")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# 🏆 7. Selección Automática del Mejor Modelo

In [ ]:
mejor = df_resultados.sort_values(by="Accuracy", ascending=False).iloc[0]
mejor

El **mejor modelo** según Accuracy es:

In [ ]:
print(f"🏆 Mejor Modelo: {mejor['Modelo']} con Accuracy = {mejor['Accuracy']:.4f}")

# 🔍 8. Matriz de Confusión del Mejor Modelo

In [ ]:
modelo_final = modelos[mejor['Modelo']]
pred_final = modelo_final.predict(X_test)
cm = confusion_matrix(y_test, pred_final)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title(f"Matriz de Confusión — {mejor['Modelo']}")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.show()

# 📝 9. Conclusiones

- Se compararon 5 modelos de clasificación.
- El modelo con mayor rendimiento fue **{mejor['Modelo']}**, con un Accuracy de **{mejor['Accuracy']:.4f}**.
- Este notebook permite validar si el modelo utilizado en Kedro es efectivamente el más óptimo.
- Si un modelo alternativo supera al actual, puede integrarse fácilmente al pipeline.
